<a href="https://colab.research.google.com/github/ReduanulRumel/Quiz-Management-System/blob/main/PythonProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import random
import time
from datetime import datetime

def read_questions():
    questions = []
    try:
        with open('questions.csv', mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                questions.append(row)
    except FileNotFoundError:
        pass
    return questions


def write_questions(questions):
    with open('questions.csv', mode='w', newline='') as file:
        fieldnames = ['Question ID', 'Question Text', 'Option A', 'Option B', 'Option C', 'Option D', 'Correct Answer', 'Marks']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for question in questions:
            writer.writerow(question)

def add_question():
    question_id = input("Enter question ID: ")
    question_text = input("Enter question text: ")
    option_a = input("Enter option A: ")
    option_b = input("Enter option B: ")
    option_c = input("Enter option C: ")
    option_d = input("Enter option D: ")
    correct_answer = input("Enter correct answer (A/B/C/D): ")
    marks = int(input("Enter marks for this question: "))

    question = {
        'Question ID': question_id,
        'Question Text': question_text,
        'Option A': option_a,
        'Option B': option_b,
        'Option C': option_c,
        'Option D': option_d,
        'Correct Answer': correct_answer,
        'Marks': marks
    }

    questions = read_questions()
    questions.append(question)
    write_questions(questions)
    print("Question added successfully!")

def update_question():
    question_id = input("Enter the question ID to update: ")
    questions = read_questions()
    for question in questions:
        if question['Question ID'] == question_id:
            question['Question Text'] = input(f"Enter new text for '{question['Question Text']}': ")
            question['Option A'] = input(f"Enter new Option A: ")
            question['Option B'] = input(f"Enter new Option B: ")
            question['Option C'] = input(f"Enter new Option C: ")
            question['Option D'] = input(f"Enter new Option D: ")
            question['Correct Answer'] = input(f"Enter new correct answer (A/B/C/D): ")
            question['Marks'] = int(input(f"Enter new marks for this question: "))
            write_questions(questions)
            print("Question updated successfully!")
            return
    print("Question not found.")


def delete_question():
    question_id = input("Enter the question ID to delete: ")
    questions = read_questions()
    updated_questions = [q for q in questions if q['Question ID'] != question_id]
    if len(updated_questions) < len(questions):
        write_questions(updated_questions)
        print("Question deleted successfully!")
    else:
        print("Question not found.")


def conduct_quiz():
    questions = read_questions()
    if not questions:
        print("No questions available.")
        return

    participant_name = input("Enter your name: ")
    random.shuffle(questions)
    score = 0
    total_marks = 0
    start_time = time.time()

    question_time_limit = 10

    for question in questions:
        print(f"Question: {question['Question Text']}")
        print(f"A. {question['Option A']}")
        print(f"B. {question['Option B']}")
        print(f"C. {question['Option C']}")
        print(f"D. {question['Option D']}")


        question_start_time = time.time()


        answer = None
        while time.time() - question_start_time < question_time_limit:
            answer = input("Your answer (A/B/C/D): ").upper()
            if answer in ['A', 'B', 'C', 'D']:
                break
            else:
                print("Invalid input, please enter A, B, C, or D.")

        if answer is None:
            print(f"Time's up! The correct answer was {question['Correct Answer']}.")
        elif answer == question['Correct Answer']:
            print("Correct!")
            score += int(question['Marks'])
        else:
            print(f"Incorrect! The correct answer is {question['Correct Answer']}.")

        total_marks += int(question['Marks'])

    time_taken = time.time() - start_time
    print(f"Quiz completed! Time taken: {round(time_taken, 2)} seconds.")

    percentage = (score / total_marks) * 100
    grade = 'F'
    if percentage >= 90:
        grade = 'A'
    elif percentage >= 80:
        grade = 'B'
    elif percentage >= 70:
        grade = 'C'
    elif percentage >= 50:
        grade = 'D'

    print(f"Your total score: {score}/{total_marks}")
    print(f"Your percentage: {round(percentage, 2)}%")
    print(f"Your grade: {grade}")


    with open('quiz_results.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([participant_name, datetime.now().strftime('%Y-%m-%d %H:%M:%S'), score])

    print("Quiz results saved!")


def display_leaderboard():
    try:
        with open('quiz_results.csv', mode='r') as file:
            reader = csv.reader(file)
            results = list(reader)
            results.sort(key=lambda x: int(x[2]), reverse=True)

            print("\nLeaderboard:")
            for i, result in enumerate(results[:10]):
                print(f"{i+1}. {result[0]} - Score: {result[2]} ({result[1]})")
    except FileNotFoundError:
        print("No quiz results available.")


def export_report():
    report_format = input("Enter the report format (CSV/TXT): ").upper()
    if report_format == 'CSV':
        try:
            with open('quiz_results.csv', mode='r') as file:
                reader = csv.reader(file)
                results = list(reader)
                with open('quiz_report.csv', mode='w', newline='') as report_file:
                    writer = csv.writer(report_file)
                    writer.writerows(results)
            print("Report exported successfully!")
        except FileNotFoundError:
            print("No quiz results available.")
    elif report_format == 'TXT':
        try:
            with open('quiz_results.csv', mode='r') as file:
                reader = csv.reader(file)
                results = list(reader)
                with open('quiz_report.txt', mode='w') as report_file:
                    for result in results:
                        report_file.write(f"Participant: {result[0]}, Score: {result[2]}, Date: {result[1]}\n")
            print("Report exported successfully!")
        except FileNotFoundError:
            print("No quiz results available.")
    else:
        print("Invalid format.")

def main_menu():
    while True:
        print("\nQuiz Management System")
        print("1. Add Question")
        print("2. Update Question")
        print("3. Delete Question")
        print("4. Start Quiz")
        print("5. View Leaderboard")
        print("6. Export Reports")
        print("7. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            add_question()
        elif choice == '2':
            update_question()
        elif choice == '3':
            delete_question()
        elif choice == '4':
            conduct_quiz()
        elif choice == '5':
            display_leaderboard()
        elif choice == '6':
            export_report()
        elif choice == '7':
            print("Exiting the system...")
            break
        else:
            print("Invalid choice, please try again.")


if __name__ == "__main__":
    main_menu()
